# Explore 1vsAll results

Andrew E. Davidson 
aedavids@ucsc.edu 
3/21/24  

Copyright (c) 2020-2023, Regents of the University of California All rights reserved. https://polyformproject.org/licenses/noncommercial/1.0.0


our basic CreateSignatureGeneConfig algorithm is a varation of 
1. genes where padj <= 0.001
2. lfc >= 2.0
3. sort by baseMean
4. take top N

This method select biomarkers that are biologically and statitically signfigants. How ever it is biased toward genes that have very strong signals

What we really want are the best biomarkers ie the highest lfc that are biologically signfigant. Alternative algorithm

A. genes where padj <= 0.001  
B. lfc >= 2.0  
C. calculate baseMean median  
D. select genes where baseMean > baseMean median  
E. sort by lfc  
F. take top N  


## Do we get better deconvolution results?

## Do we get better plasma classifier results?

**results**  
looking at top 10 for Whole_Blood, Lung, LUAD, and LUSC. it looks like Whole_Blood will work better. We get biomarkers with much larger LFC values. 

With the old approach all, the lung releated types have a small number of over laping genes. Under the new approach only lung and LUAD share genes. The LFC values for LUAD are much lower than for Lung. Not obvious if this will work better or not

**todo**  
run deconvolution pipeline using new algorithm and compare with old (BestSignatureGeneConfig)


In [1]:
import ipynbname
from IPython.display import display
import numpy as np
import os
import pandas as pd
import sys

notebookName = ipynbname.name()
notebookPath = ipynbname.path()
notebookDir = os.path.dirname(notebookPath)

/data/tmp/ipykernel_3123170/3124537099.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# setting the python path allows us to run python scripts from using
# the CLI. 
ORIG_PYTHONPATH = os.environ['PYTHONPATH']

deconvolutionModules = notebookPath.parent.joinpath("../python/")
print("deconvolutionModules: {}\n".format(deconvolutionModules))

PYTHONPATH = ORIG_PYTHONPATH + f':{deconvolutionModules}'
print("PYTHONPATH: {}\n".format(PYTHONPATH))

#intraExtraRNA_POCModules=notebookPath.parent.joinpath("../../../python/src")
intraExtraRNA_POCModules=notebookPath.parent.joinpath("../../intraExtraRNA_POC/python/src")
print("intraExtraRNA_POCModules: {}\n".format(intraExtraRNA_POCModules))

PYTHONPATH = PYTHONPATH + f':{intraExtraRNA_POCModules}'
print("PYTHONPATH: {}\n".format(PYTHONPATH))

os.environ["PYTHONPATH"] = PYTHONPATH
PYTHONPATH = os.environ["PYTHONPATH"]
print("PYTHONPATH: {}\n".format(PYTHONPATH))

# to be able to import our local python files we need to set the sys.path
# https://stackoverflow.com/a/50155834
sys.path.append( str(deconvolutionModules) )
sys.path.append( str(intraExtraRNA_POCModules) )
print("\nsys.path:\n{}\n".format(sys.path))

deconvolutionModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/../python

intraExtraRNA_POCModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/../../intraExtraRNA_POC/python/src

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/../python:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/../../intraExtraRNA_POC/python/src

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/../python:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/../../intraExtraRNA_POC/python/src


sys.path:
['/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jup

In [3]:
# import local python files
from pipeline.dataFactory.driver import _countExtraHeaderLines
from analysis.bestSignatureGeneConfig import BestSignatureGeneConfig

In [4]:
rootDir="/private/groups/kimlab/GTEx_TCGA"
colData=f"{rootDir}/groupbyGeneTrainingSets/GTEx_TCGA_TrainColData.csv"
countData=f"{rootDir}/groupbyGeneTrainingSets/GTEx_TCGA_TrainGroupby.csv"

# 
# deseqResultsDir is the directory of results files that will be passed to 
# SignatureGenes.findGenes()
#
deseqResultsDir=f"{rootDir}/1vsAll"

In [5]:
deseqResultFiles = ! ls $deseqResultsDir/*results
assert len(deseqResultFiles) == 83, "ERROR missing results file"
print(f'{deseqResultFiles[0]}')

/private/groups/kimlab/GTEx_TCGA/1vsAll/ACC_vs_all.results


In [6]:
def loadResults( resultsFiles : list[str] ) -> dict[str, pd.DataFrame] :
    retDict = dict()
    suffixLength = len("_vs_all.results")
    for resultsFile in resultsFiles:
        numRowsToSkip = _countExtraHeaderLines(resultsFile)
        # logger.info(f"runSelectGenesOfInterest() csgpFile: {csgpFile}")
        # logger.info(f"runSelectGenesOfInterest() numRowsToSkip: {numRowsToSkip}")
        deseqDF = pd.read_csv(resultsFile, skiprows=numRowsToSkip)

        fileName = resultsFile.split("/")[-1]
        key = fileName[:-suffixLength]
        print(f'{key}')
        # signatureGenesDF = signatureGeneConfig.findGenes(deseqDF, fileName)   

        # outFilePath = outDir.joinpath( fileName )

        # if save and not signatureGenesDF.empty :
        #     signatureGenesDF.to_csv(outFilePath, index=False)
        #     logger.warning("saved to file: {}".format(outFilePath))
        # else:
        #     logger.warning(f'{csgpFile} has no signature genes')

        retDict[key] = deseqDF
        # retOutFileList.append(outFilePath)

    return retDict

resultsDataFrameDict = loadResults(deseqResultFiles)

ACC
Adipose_Subcutaneous
Adipose_Visceral_Omentum
Adrenal_Gland
Artery_Aorta
Artery_Coronary
Artery_Tibial
Bladder
BLCA
Brain_Amygdala
Brain_Anterior_cingulate_cortex_BA24
Brain_Caudate_basal_ganglia
Brain_Cerebellar_Hemisphere
Brain_Cerebellum
Brain_Cortex
Brain_Frontal_Cortex_BA9
Brain_Hippocampus
Brain_Hypothalamus
Brain_Nucleus_accumbens_basal_ganglia
Brain_Putamen_basal_ganglia
Brain_Spinal_cord_cervical_c-1
Brain_Substantia_nigra
BRCA
Breast_Mammary_Tissue
Cells_Cultured_fibroblasts
Cells_EBV-transformed_lymphocytes
Cervix_Endocervix
CESC
CHOL
COAD
Colon_Sigmoid
Colon_Transverse
DLBC
ESCA
Esophagus_Gastroesophageal_Junction
Esophagus_Mucosa
Esophagus_Muscularis
GBM
Heart_Atrial_Appendage
Heart_Left_Ventricle
HNSC
KICH
Kidney_Cortex
KIRC
KIRP
LGG
LIHC
Liver
LUAD
Lung
LUSC
MESO
Minor_Salivary_Gland
Muscle_Skeletal
Nerve_Tibial
Ovary
OV
PAAD
Pancreas
PCPG
Pituitary
PRAD
Prostate
READ
SARC
SKCM
Skin_Not_Sun_Exposed_Suprapubic
Skin_Sun_Exposed_Lower_leg
Small_Intestine_Terminal_Ileum


In [7]:
print(f"{resultsDataFrameDict['Whole_Blood'].shape}")
resultsDataFrameDict['Whole_Blood'].head()

(74777, 7)


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,(ATGAA)n,127.590865,-4.162891,0.083563,-49.817279,0.0,0.0
1,(CAAC)n,5.138829,5.790844,0.102441,56.528728,0.0,0.0
2,(CATCT)n,82.515068,-5.106976,0.110907,-46.047376,0.0,0.0
3,(CTCCTC)n,4.535696,4.428033,0.089865,49.274449,0.0,0.0
4,(GAAG)n,2.386262,2.878263,0.066669,43.172376,0.0,0.0


In [8]:
def selectGenesMedian(df : pd.DataFrame, topN : int) -> pd.DataFrame :
    '''
    new algorithm
    '''
    selectPadjRows = df.loc[:, "padj"] <= 0.001
    df = df.loc[selectPadjRows, :]
    print(f'\nselectPadjRows {sum(selectPadjRows)} {df.shape}' ) 

    selectLFCRows = df.loc[:, "log2FoldChange"].abs() > 2.0
    df = df.loc[selectLFCRows, :]
    print(f'\nselectLFCRows {sum(selectLFCRows)} {df.shape}')

    baseMeanMedian = df.loc[:, "baseMean"].median()
    print(f'baseMeanMedian :{baseMeanMedian}')

    selectAboveMedianRows = df.loc[:, "baseMean"] >= baseMeanMedian
    df = df.loc[selectAboveMedianRows, :]
    print(f'\nselectAboveMedianRows {sum(selectAboveMedianRows)} {df.shape}')

    biomarkersDF = df.sort_values(by="log2FoldChange", ascending=False)
    # # the results for Whole_Blood are suprising make sure abs() works as expected
    # we expect there to be some abs(LFC) < -2 and padj <= 0.001 , baseMean > median 
    # that filtered out because becuase their LFC is low compared to "best"
    # print(f'aedwip biomarkersDF are there neg lfc values')
    # print(f'{biomarkersDF.tail()}')

    return biomarkersDF.head(n=topN)

dfMedian = selectGenesMedian(resultsDataFrameDict['Whole_Blood'], topN=10)
dfMedian


selectPadjRows 31247 (31247, 7)

selectLFCRows 11455 (11455, 7)
baseMeanMedian :102.795064959949

selectAboveMedianRows 5728 (5728, 7)


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
3029,HBA2,939295.036781,11.396289,0.106056,107.455747,0.0,0.0
3030,HBB,985081.569770,11.377987,0.110899,102.597364,0.0,0.0
3028,HBA1,384428.792720,11.333315,0.107194,105.727071,0.0,0.0
816,ALAS2,3853.245600,10.966347,0.122523,89.504673,0.0,0.0
392,AC104389.6,1032.067944,10.650671,0.215678,49.382235,0.0,0.0
3034,HBM,373.003021,10.629039,0.152953,69.492163,0.0,0.0
3031,HBD,1044.466292,10.179380,0.140412,72.496635,0.0,0.0
1969,CXCR1,3110.515693,10.057911,0.110750,90.816569,0.0,0.0
3033,HBG2,1115.922931,10.018484,0.127201,78.761075,0.0,0.0
629,AHSP,286.554038,9.909376,0.143479,69.065175,0.0,0.0


In [9]:
tmpDir = f'{notebookDir}/{notebookName}.tmp'

bsgc = BestSignatureGeneConfig(
    dataSetName="dataSetName_str", 
    design="design_str", 
    padjThreshold=0.001, 
    lfcThreshold=2.0, 
    n=10, 
    localCacheRootPath=tmpDir, 
    title="title_str"
)

dfBase = bsgc._select(resultsDataFrameDict['Whole_Blood'], "fileName_AEDWIP").head(n=10)
dfBase

,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,absLog2FoldChange
10096,MT-CO1,1.346880e+06,-2.152827,0.070782,-30.414982,3.481285e-203,1.685098e-202,2.152827
3030,HBB,9.850816e+05,11.377987,0.110899,102.597364,0.000000e+00,0.000000e+00,11.377987
9179,MT-ND4,9.479209e+05,-2.092066,0.064367,-32.502050,9.974838e-232,5.310569e-231,2.092066
3029,HBA2,9.392950e+05,11.396289,0.106056,107.455747,0.000000e+00,0.000000e+00,11.396289
9089,MT-CO3,5.908987e+05,-2.059607,0.062952,-32.716936,8.970390e-235,4.823090e-234,2.059607
4243,MT-ND2,5.499334e+05,-2.429095,0.063377,-38.327863,0.000000e+00,0.000000e+00,2.429095
9331,MT-ATP6,5.114564e+05,-2.079334,0.064703,-32.136498,1.363746e-226,7.142275e-226,2.079334
8195,MT-CYB,5.009196e+05,-2.216252,0.063129,-35.106642,5.337883e-270,3.183061e-269,2.216252
3822,LSU-rRNA,4.326240e+05,3.302824,0.075455,43.771867,0.000000e+00,0.000000e+00,3.302824
9906,MT-ND1,4.252936e+05,-2.063931,0.066907,-30.847775,5.999987e-209,2.959961e-208,2.063931


In [10]:
# are there common genes?
set( dfMedian.loc[:, "name"] ).intersection( dfBase.loc[:, "name"] )

{'HBA2', 'HBB'}

## do we get better genes?

check using genecard

- HBA2 Gene - Hemoglobin Subunit Alpha 2
- HBB Gene - Hemoglobin Subunit Beta
- HB* is probably Hemoglobin


- MT-CO1	Mitochondrially Encoded Cytochrome C Oxidase I
- MT-ND4 Mitochondrially Encoded NADH:Ubiquinone Oxidoreductase Core Subunit 4	

# Check Lung, LUAD, and LUSC


In [11]:
oldLungDF = bsgc._select(resultsDataFrameDict['Lung'], "fileName_AEDWIP").head(n=10)
oldLungSet = set( oldLungDF.loc[:, "name"] )
oldLungDF

,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,absLog2FoldChange
843,MT-RNR2,774541.966032,-2.377409,0.088118,-26.979831,2.549070e-160,1.203227e-158,2.377409
383,MT-RNR1,140332.123897,-2.992970,0.093822,-31.900395,2.636597e-223,2.735401e-221,2.992970
1912,DES,89881.666167,-3.663429,0.167908,-21.818133,1.561047e-105,3.250943e-104,3.663429
5205,MYH11,76286.185370,-2.251269,0.145358,-15.487774,4.195663e-54,3.210738e-53,2.251269
514,KRT5,48084.270387,-5.994798,0.197728,-30.318368,6.565490e-202,5.078885e-200,5.994798
2341,TPM2,43129.627581,-2.324521,0.113092,-20.554192,7.059588e-94,1.200884e-92,2.324521
749,KRT6A,36638.110998,-6.116929,0.219756,-27.835047,1.634223e-170,8.680773e-169,6.116929
133,A2M,34166.126479,3.092962,0.083189,37.179964,1.438447e-302,4.276588e-300,3.092962
3296,CRYAB,21594.014674,-2.198742,0.119609,-18.382788,1.804529e-75,2.180486e-74,2.198742
606,DSP,21273.622046,-4.038954,0.137773,-29.315996,6.486265e-189,4.257106e-187,4.038954


In [12]:
newLungDF = selectGenesMedian(resultsDataFrameDict['Lung'], topN=10)
newLungSet = set(newLungDF.loc[:, "name"] )
newLungDF


selectPadjRows 24516 (24516, 7)

selectLFCRows 4037 (4037, 7)
baseMeanMedian :17.3175849432978

selectAboveMedianRows 2019 (2019, 7)


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
93,SFTPA1,5205.334645,11.227874,0.216943,51.755036,0.000000e+00,0.000000e+00
94,SFTPA2,7067.487718,10.759414,0.178667,60.220576,0.000000e+00,0.000000e+00
96,SFTPC,4504.007216,10.476578,0.190973,54.858980,0.000000e+00,0.000000e+00
91,SCGB1A1,460.654113,9.778221,0.230285,42.461428,0.000000e+00,0.000000e+00
63,MS4A15,68.376882,9.528338,0.183990,51.787233,0.000000e+00,0.000000e+00
95,SFTPB,14421.444640,9.215714,0.199454,46.204810,0.000000e+00,0.000000e+00
168,SCGB3A2,419.178498,8.864025,0.243186,36.449560,6.990594e-291,1.647919e-288
162,ALPP,68.865090,8.795942,0.240238,36.613467,1.746281e-293,4.268103e-291
65,NAPSA,1863.020676,7.732604,0.130391,59.303341,0.000000e+00,0.000000e+00
273,SNTN,22.040783,7.287121,0.215156,33.868978,1.907623e-251,2.773641e-249


In [13]:
oldLUADDF = bsgc._select(resultsDataFrameDict['LUAD'], "fileName_AEDWIP").head(n=10)
oldLUADSet= set( oldLUADDF.loc[:, "name"] )
oldLUADDF

,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,absLog2FoldChange
261,DES,89881.666167,-4.153133,0.180200,-23.047413,1.561467e-117,2.267881e-115,4.153133
388,CLU,56070.788880,-2.273153,0.107877,-21.071755,1.444750e-98,1.413293e-96,2.273153
1383,KRT6A,36638.110998,-3.608496,0.236553,-15.254489,1.536931e-52,4.225783e-51,3.608496
2345,IGHA1,25987.995442,2.265104,0.173259,13.073548,4.663817e-39,7.564886e-38,2.265104
3,CRYAB,21594.014674,-5.135958,0.127156,-40.390908,0.000000e+00,0.000000e+00,5.135958
928,KRT17,19620.982687,-3.223883,0.188361,-17.115405,1.139206e-65,4.666331e-64,3.223883
677,RGS5,18048.226869,-2.193606,0.118217,-18.555835,7.316229e-77,4.103218e-75,2.193606
329,NDRG2,17790.572037,-2.249253,0.102673,-21.907055,2.225110e-106,2.565822e-104,2.249253
1151,SERPINA3,14604.186235,-2.506750,0.155628,-16.107276,2.267954e-58,7.491534e-57,2.506750
174,SFTPB,14421.444640,6.019867,0.241647,24.911810,5.541549e-137,1.204986e-134,6.019867


In [14]:
oldLUSCDF = bsgc._select(resultsDataFrameDict['LUSC'], "fileName_AEDWIP").head(n=10)
oldLUSCSet= set( oldLUSCDF.loc[:, "name"] )
oldLUSCDF

,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,absLog2FoldChange
53,DES,89881.666167,-5.791090,0.183208,-31.609387,2.743427e-219,2.030695e-216,5.791090
102,CLU,56070.788880,-3.161935,0.109496,-28.877267,2.304252e-183,8.942062e-181,3.161935
626,GPX3,40969.907575,-2.509150,0.120399,-20.840239,1.869043e-96,1.191508e-94,2.509150
283,FHL1,28068.059244,-2.877531,0.118019,-24.382012,2.654020e-131,3.735346e-129,2.877531
1289,APOE,22484.798309,-2.267948,0.129077,-17.570452,4.148511e-69,1.285428e-67,2.267948
411,CRYAB,21594.014674,-2.961866,0.130609,-22.677278,7.509238e-114,7.285237e-112,2.961866
195,RGS5,18048.226869,-3.104350,0.120079,-25.852492,2.281066e-147,4.651862e-145,3.104350
1798,APOD,17203.670074,-2.084074,0.129336,-16.113631,2.046466e-58,4.546932e-57,2.084074
265,PDK4,16773.120205,-3.119649,0.126792,-24.604445,1.132136e-133,1.701226e-131,3.119649
834,ADH1B,15806.768045,-3.190317,0.162393,-19.645703,6.291979e-86,3.011936e-84,3.190317


In [15]:
newLUADDF = selectGenesMedian(resultsDataFrameDict['LUAD'], topN=10)
newLUADSet= set( newLUADDF.loc[:, "name"] )
newLUADDF


selectPadjRows 18908 (18908, 7)

selectLFCRows 1572 (1572, 7)
baseMeanMedian :29.3865804899171

selectAboveMedianRows 786 (786, 7)


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
1362,BPIFA1,273.113947,7.135143,0.466288,15.302020,7.412007e-53,2.069326e-51
246,SCGB3A2,419.178498,6.829563,0.293545,23.265795,9.845214e-120,1.516761e-117
7,NAPSA,1863.020676,6.808249,0.160603,42.391826,0.000000e+00,0.000000e+00
377,GGTLC1,39.187032,6.748057,0.317720,21.238996,4.166979e-100,4.194869e-98
168,IL37,66.152261,6.235315,0.249544,24.986833,8.500228e-138,1.913960e-135
334,SFTPA1,5205.334645,6.175151,0.283162,21.807808,1.956263e-105,2.222139e-103
133,SFTPA2,7067.487718,6.158545,0.236783,26.009190,3.898078e-149,1.106967e-146
216,SFTPC,4504.007216,6.029687,0.250870,24.035119,1.194581e-127,2.094810e-125
174,SFTPB,14421.444640,6.019867,0.241647,24.911810,5.541549e-137,1.204986e-134
165,CALCA,358.195196,5.925402,0.235614,25.148787,1.457027e-139,3.340015e-137


In [16]:
newLUSCDF = selectGenesMedian(resultsDataFrameDict['LUSC'], topN=10)
newLUSCSet= set( newLUSCDF.loc[:, "name"] )
newLUSCDF


selectPadjRows 20480 (20480, 7)

selectLFCRows 1716 (1716, 7)
baseMeanMedian :20.445675984614198

selectAboveMedianRows 858 (858, 7)


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
708,LINC01206,74.404086,5.524838,0.272599,20.267276,2.501248e-91,1.410118e-89
849,SOST,152.792635,5.096144,0.260232,19.583066,2.156421e-85,1.014051e-83
105,NMRAL2P,147.349852,4.411344,0.154763,28.503808,1.050811e-178,3.962451e-176
3318,UGT1A7,250.840906,4.030537,0.303638,13.274134,3.270297e-40,3.938447e-39
3456,ADH7,426.779623,3.972077,0.303427,13.090696,3.721839e-39,4.303316e-38
7258,AL033397.1,36.321306,3.784155,0.401060,9.435385,3.895626e-21,2.145090e-20
11835,CT45A1,28.213118,3.652643,0.536049,6.814014,9.491241e-12,3.205258e-11
164,AC090004.2,82.700060,3.649634,0.136350,26.766584,7.917693e-158,1.918049e-155
2065,TREML3P,34.254539,3.508796,0.226551,15.487861,4.189988e-54,8.106389e-53
10125,MAGEA4,213.122638,3.422866,0.443517,7.717550,1.185872e-14,4.681067e-14


## was there an over lap with old biomarkers?
yes! ideally we want our biomarkers to be disjoint. ie we want the choose biomarkers that are
far apart (euclidian distance) from each other. This should make the data more seperable.

In [17]:
oldLungSet.intersection( oldLUADSet)

{'CRYAB', 'DES', 'KRT6A'}

In [18]:
oldLungSet.intersection( oldLUSCSet)

{'CRYAB', 'DES'}

In [19]:
oldLUADSet.intersection(oldLUSCSet)

{'CLU', 'CRYAB', 'DES', 'RGS5'}

## Is there any over lap in biomarkers between old and new?
We expect interesection to be empty

In [20]:
oldLungSet.intersection( newLungSet )

set()

In [21]:
oldLUADSet.intersection( newLUADSet )

{'SFTPB'}

In [22]:
oldLUSCSet.intersection( newLUSCSet )

set()

In [23]:
newLUADSet.intersection(newLUSCSet)

set()

## Is there an overlap with new biomarkers?
we expect the intersection to be empty

In [24]:
newLungNewLUADIntersection = newLungSet.intersection( newLUADSet)
print(newLungNewLUADIntersection)
print(newLungSet.intersection( newLUSCSet))
print(newLUADSet.intersection( newLUSCSet)) 

{'NAPSA', 'SFTPC', 'SFTPA2', 'SFTPB', 'SFTPA1', 'SCGB3A2'}
set()
set()


In [25]:
selectRows = newLungDF.loc[:,"name"].isin(newLungNewLUADIntersection)
newLungDF.loc[selectRows, ["name", "baseMean", "log2FoldChange"] ].sort_values(by="name")

,name,baseMean,log2FoldChange
65,NAPSA,1863.020676,7.732604
168,SCGB3A2,419.178498,8.864025
93,SFTPA1,5205.334645,11.227874
94,SFTPA2,7067.487718,10.759414
95,SFTPB,14421.444640,9.215714
96,SFTPC,4504.007216,10.476578


In [26]:
selectRows = newLUADDF.loc[:,"name"].isin(newLungNewLUADIntersection)
newLUADDF.loc[selectRows, ["name", "baseMean", "log2FoldChange"]].sort_values(by="name")

,name,baseMean,log2FoldChange
7,NAPSA,1863.020676,6.808249
246,SCGB3A2,419.178498,6.829563
334,SFTPA1,5205.334645,6.175151
133,SFTPA2,7067.487718,6.158545
174,SFTPB,14421.444640,6.019867
216,SFTPC,4504.007216,6.029687
